## Import necessary packages

In [1]:
import sys
sys.path.append('../')

import hdf5plugin
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import pandas as pd
import scanpy as sc
import pickle
import anndata as ad
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from torch.optim.lr_scheduler import _LRScheduler, ReduceLROnPlateau
from tqdm import tqdm
from copy import deepcopy
from dance.utils import set_seed
from scipy.sparse import csr_matrix
from CellPLM.utils.eval import downstream_eval
from CellPLM.utils.data import XDict, clean_batches, stratified_sample_genes_by_sparsity
from CellPLM.model import OmicsFormer
from CellPLM.head import setup_head

def create_sparse_tensor(x):
    return torch.sparse_csr_tensor(x.indptr, x.indices, x.data, (x.shape[0], x.shape[1])).to_sparse().float().coalesce()

/egr/research-dselab/wenhongz/miniconda3/envs/cellplm/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Specify important parameters before getting started

In [2]:
DATASET = 'MS' # 'hPancreas'
PRETRAIN = '20230926_85M'
DEVICE = 'cuda:5'

## Load Downstream Dataset

The MS dataset is contributed by [scGPT](https://github.com/bowang-lab/scGPT/blob/main/tutorials/Tutorial_Annotation.ipynb). hPancreas dataset is contributed by [TOSICA](https://github.com/JackieHanLab/TOSICA/blob/main/test/tutorial.ipynb).


In [3]:
set_seed(10)
if DATASET == 'hPancreas':
    data_train = ad.read_h5ad(f'../data/demo_train.h5ad')
    data_test = ad.read_h5ad(f'../data/demo_test.h5ad')
    train_num = data_train.shape[0]
    data = ad.concat([data_train, data_test])
    data.X = csr_matrix(data.X)
    data.obs['celltype'] = data.obs['Celltype']
    data.obs['batch'] = 0

    import mygene
    mg = mygene.MyGeneInfo()
    data.var.index = mg.querymany(data.var.index.tolist(), scopes='symbol', fields='ensembl.gene', as_dataframe=True, species='human').reset_index().drop_duplicates(subset='query')['ensembl.gene'].fillna('0').tolist()
    data.var_names_make_unique()

elif DATASET == 'MS':
    data_train = ad.read_h5ad(f'../data/c_data.h5ad')
    data_test = ad.read_h5ad(f'../data/filtered_ms_adata.h5ad')
    data_train.var = data_train.var.set_index('index_column')
    data_test.var = data_test.var.set_index('index_column')
    train_num = data_train.shape[0]
    data = ad.concat([data_train, data_test])
    data.obs['batch'] = 0
    data.var_names_make_unique()

with (open(f"../ckpt/{PRETRAIN}.config.pkl", "rb")) as openfile:
    config = pickle.load(openfile)
pretrain_gene_list = config['gene_list']
gene_list = data.var.index.tolist()
gene_list = [x for x in gene_list if x in pretrain_gene_list]
data = data[:, gene_list]
print(data.shape)

order = np.arange(data.shape[0])
labels = LabelEncoder().fit_transform(data.obs['celltype'])
batch_labels = LabelEncoder().fit_transform(data.obs['batch'])
out_dim = len(np.unique(labels))

train_idx = []
valid_idx = []
test_idx = []

for batch in tqdm(range(batch_labels.max() + 1)):
    tr = torch.randperm(train_num).long()
    temp = torch.zeros([data[batch_labels == batch].shape[0]])
    temp[tr[:int(train_num*0.9)]] = 1
    train_idx.append(temp.bool())
    temp = torch.zeros([data[batch_labels == batch].shape[0]])
    temp[tr[int(train_num*0.9):train_num]] = 1
    valid_idx.append(temp.bool())
    temp = torch.zeros([data[batch_labels == batch].shape[0]])
    temp[train_num:] = 1
    test_idx.append(temp.bool())


[INFO][2023-10-10 15:31:12,294][dance][set_seed] Setting global random seed to 10


(21312, 2763)


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 12.65it/s]


## Overwrite parts of the pretrain config for fine-tuning
These hyperparameters are recommended for general purpose. We did not tune it for individual datasets.

In [4]:
config['es'] = 200
config['lr'] = 5e-3
config['wd'] = 1e-7
config['scheduler'] = 'plat'
config['drop_node_rate'] = 0.3
config['dec_layers'] = 1
config['model_dropout'] = 0.5
config['mask_node_rate'] = 0.75
config['mask_feature_rate'] = 0.25
config['dec_mod'] = 'mlp'
config['latent_mod'] = 'ae'
config['epochs'] = 2000
config['head_type'] = 'annotation'
config['out_dim'] = out_dim
config['max_batch_size'] = 70000
config['hvg'] = 3000
config['batch_num'] = batch_labels.max() + 1
config['patience'] = 25

## Fine-tuning

Preprocess data for CellPLM.

In [5]:
seq_list = []
batch_list = []
coord_list = []
label_list = []
data_hvg = data.copy()
if config['hvg'] < data_hvg.shape[1]:
    sc.pp.highly_variable_genes(data_hvg, n_top_genes=config['hvg'], subset=True, flavor='seurat_v3')
gene_list = data_hvg.var.index.tolist()
for batch in tqdm(range(batch_labels.max() + 1)):
    x = data_hvg[batch_labels == batch].X.astype(float)
    seq_list.append(create_sparse_tensor(x))
    batch_list.append(torch.from_numpy(batch_labels[batch_labels == batch]))
    coord_list.append(torch.zeros(x.shape[0], 2) - 1)
    label_list.append(torch.from_numpy(labels[batch_labels == batch]).float())

  0%|                                                                                                                                     | 0/1 [00:00<?, ?it/s]/tmp/ipykernel_933360/2882960297.py:25: UserWarning: Sparse CSR tensor support is in beta state. If you miss a functionality in the sparse tensor support, please submit a feature request to https://github.com/pytorch/pytorch/issues. (Triggered internally at ../aten/src/ATen/SparseCsrTensorImpl.cpp:54.)
  return torch.sparse_csr_tensor(x.indptr, x.indices, x.data, (x.shape[0], x.shape[1])).to_sparse().float().coalesce()
100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00,  8.40it/s]


Prepare pretrained model.

In [6]:
device = torch.device(DEVICE)
model = OmicsFormer(**config)
pretrained_file = f'../ckpt/{PRETRAIN}.best.ckpt'
pretrained_model_dict = torch.load(pretrained_file)['model_state_dict']
model_dict = model.state_dict()
pretrained_dict = {
    k: v
    for k, v in pretrained_model_dict.items()
    if k in model_dict and v.shape == model_dict[k].shape
}
model_dict.update(pretrained_dict)
model.load_state_dict(model_dict)
model.to(device)

OmicsFormer(
  (embedder): OmicsEmbeddingLayer(
    (act): ReLU()
    (norm0): GroupNorm(4, 1024, eps=1e-05, affine=True)
    (dropout): Dropout(p=0.5, inplace=False)
    (extra_linear): Sequential(
      (0): Linear(in_features=1024, out_features=1024, bias=True)
      (1): ReLU()
      (2): Dropout(p=0.5, inplace=False)
      (3): GroupNorm(4, 1024, eps=1e-05, affine=True)
    )
    (pe_enc): Sinusoidal2dPE(
      (pe_enc): Embedding(10000, 1024)
    )
    (feat_enc): OmicsEmbedder()
  )
  (mask_model): MaskBuilder()
  (encoder): TransformerEncoder(
    (layers): ModuleList(
      (0): FlowformerLayer(
        (self_attn): Flow_Attention(
          (query_projection): Linear(in_features=1024, out_features=1024, bias=True)
          (key_projection): Linear(in_features=1024, out_features=1024, bias=True)
          (value_projection): Linear(in_features=1024, out_features=1024, bias=True)
          (out_projection): Linear(in_features=1024, out_features=1024, bias=True)
          (drop

## Start training
Estimated training time is 10 minutes. Estimated early stop epoch is around 500.

In [7]:
optim = torch.optim.AdamW([
    {'params': list(model.embedder.parameters()), 'lr': config['lr']*0.1,
     'weight_decay': 1e-10},
    {'params': list(model.encoder.parameters()) + list(model.head.parameters()) + list(model.latent.parameters()), 'lr': config['lr'],
     'weight_decay': config['wd']},
])

if config['scheduler'] == 'plat':
    scheduler = ReduceLROnPlateau(optim, 'min', patience=config['patience'], factor=0.95)
    
    
eval_dict = {}
eval_dict['num_classes'] = len(np.unique(labels))
train_loss = []
valid_loss = []
valid_metric = []
test_metric = []
final_test = -1
final_epoch = -1

for epoch in tqdm(range(config['epochs'])):
    epoch_loss = []
    train_scores = []
    model.train()

    if epoch<30 and config['scheduler'] != 'cos':
        for param_group in optim.param_groups[1:]:
            param_group['lr'] = config['lr'] * (epoch+1)/30

    for i in range(len(seq_list)):
        idx = torch.arange(batch_list[i].shape[0]).long()
        input_dict = {
            'coord': coord_list[i].to(device),#[cur].to(device),
            'label': label_list[i].to(device),#[cur].to(device),
            'loss_mask': train_idx[i].to(device).bool(),#[cur].to(device).bool(),
        }
        input_dict['x_seq'] = seq_list[i].to(device)
        x_dict = XDict(input_dict)
        out_dict, loss = model(x_dict, gene_list)
        with torch.no_grad():
            train_scores.append(downstream_eval('annotation', out_dict['pred'], out_dict['label'], **eval_dict))

        optim.zero_grad()
        loss.backward()
        nn.utils.clip_grad_norm_(model.parameters(), 2.0)
        optim.step()
        epoch_loss.append(loss.item())

        if config['scheduler'] == 'cos':
            scheduler.step()

        elif config['scheduler'] == 'plat':
            scheduler.step(loss.item())
    train_scores_new = {}
    for k in train_scores[0].keys():
        train_scores_new[k] = []
        for t in train_scores:
            train_scores_new[k].append(t[k])
        train_scores_new[k] = sum(train_scores_new[k]) / len(train_scores_new[k])
    train_scores = train_scores_new
    train_loss.append(sum(epoch_loss) / len(epoch_loss))

    with torch.no_grad():
        model.eval()
        epoch_loss = []
        valid_epoch = []
        valid_pred = []
        valid_lb = []
        test_pred = []
        test_lb = []
        for i in range(len(seq_list)):
            input_dict = {
                'coord': coord_list[i].to(device),#[cur].to(device),
                'label': label_list[i].to(device),#[cur].to(device),
                'loss_mask': torch.ones([label_list[i].shape[0]]).to(device).bool(),
            }
            input_dict['x_seq'] = seq_list[i].to(device)#.index_select(0, cur.to(device))
            x_dict = XDict(input_dict)

            out_dict, loss = model(x_dict, gene_list)
            epoch_loss.append(loss.item())
            valid_pred.append(out_dict['pred'][valid_idx[i]])
            valid_lb.append(out_dict['label'][valid_idx[i]])
            test_pred.append(out_dict['pred'][test_idx[i]])
            test_lb.append(out_dict['label'][test_idx[i]])

            valid_scores = downstream_eval('annotation', torch.cat(valid_pred), torch.cat(valid_lb), **eval_dict)
            test_scores = downstream_eval('annotation', torch.cat(test_pred), torch.cat(test_lb), **eval_dict)
            valid_epoch.append(valid_scores['f1_score'])
    valid_loss.append(sum(epoch_loss) / len(epoch_loss))
    valid_metric.append(sum(valid_epoch) / len(valid_epoch))
    test_metric.append(test_scores['f1_score'])

    print(f'Epoch {epoch} | Train loss: {train_loss[-1]:.4f} | Valid loss: {valid_loss[-1]:.4f}')
    print(
        f'Train ACC: {train_scores["acc"]:.4f} | valid ACC: {valid_scores["acc"]:.4f} | test ACC: {test_scores["acc"]:.4f}')
    print(
        f'Train f1: {train_scores["f1_score"]:.4f} | valid f1: {valid_scores["f1_score"]:.4f} | test f1: {test_scores["f1_score"]:.4f}')
    print(
        f'Train pre: {train_scores["precision"]:.4f} | valid pre: {valid_scores["precision"]:.4f} | test pre: {test_scores["precision"]:.4f}')

    if max(valid_metric) == valid_metric[-1]:
        best_dict = deepcopy(model.state_dict())
        final_test = test_scores["f1_score"]
        final_epoch = epoch

    if max(valid_metric) != max(valid_metric[-config['es']:]):
        print('Early stopped.')
        break

  0%|                                                                                                                        | 1/2000 [00:13<7:22:38, 13.29s/it]

Epoch 0 | Train loss: 2.9277 | Valid loss: 2.8465
Train ACC: 0.0440 | valid ACC: 0.0695 | test ACC: 0.0559
Train f1: 0.0237 | valid f1: 0.0139 | test f1: 0.0075
Train pre: 0.0429 | valid pre: 0.0095 | test pre: 0.0096


  0%|                                                                                                                        | 2/2000 [00:16<4:00:21,  7.22s/it]

Epoch 1 | Train loss: 2.8506 | Valid loss: 2.7500
Train ACC: 0.0599 | valid ACC: 0.0804 | test ACC: 0.0627
Train f1: 0.0451 | valid f1: 0.0498 | test f1: 0.0082
Train pre: 0.0606 | valid pre: 0.0412 | test pre: 0.0326


  0%|▏                                                                                                                       | 3/2000 [00:19<2:52:33,  5.18s/it]

...

Epoch 503 | Train loss: 0.0137 | Valid loss: 0.5722
Train ACC: 0.9965 | valid ACC: 0.8533 | test ACC: 0.7873
Train f1: 0.9967 | valid f1: 0.8420 | test f1: 0.7921
Train pre: 0.9969 | valid pre: 0.8413 | test pre: 0.8134


 25%|█████████████████████████████▊                                                                                        | 505/2000 [21:35<1:06:42,  2.68s/it]

Epoch 504 | Train loss: 0.0189 | Valid loss: 0.5739
Train ACC: 0.9939 | valid ACC: 0.8533 | test ACC: 0.7869
Train f1: 0.9944 | valid f1: 0.8420 | test f1: 0.7916
Train pre: 0.9949 | valid pre: 0.8413 | test pre: 0.8151


 25%|█████████████████████████████▊                                                                                        | 505/2000 [21:37<1:04:00,  2.57s/it]

Epoch 505 | Train loss: 0.0157 | Valid loss: 0.5744
Train ACC: 0.9969 | valid ACC: 0.8493 | test ACC: 0.7847
Train f1: 0.9973 | valid f1: 0.8380 | test f1: 0.7898
Train pre: 0.9976 | valid pre: 0.8376 | test pre: 0.8126
Early stopped.


## Load best model and display final performance

In [8]:
model.load_state_dict(best_dict)
print({'final_test_f1': final_test, 'final_epoch': final_epoch, 'dataset': DATASET})

{'final_test_f1': 0.7886810302734375, 'final_epoch': 305, 'dataset': 'MS'}
